In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import math
import pickle
import Niklas_Prepare_Data as npd

In [2]:
# Prepare stirps to use them later
npd.load_no_label_data(False)

strip_1_test_no_labels  already exists
strip_2_test_no_labels  already exists
strip_3_test_no_labels  already exists
strip_4_test_no_labels  already exists
strip_5_test_no_labels  already exists
strip_6_test_no_labels  already exists
strip_7_test_no_labels  already exists
strip_8_test_no_labels  already exists
strip_9_test_no_labels  already exists
strip_10_test_no_labels  already exists
strip_11_test_no_labels  already exists
strip_12_test_no_labels  already exists
strip_13_test_no_labels  already exists
strip_14_test_no_labels  already exists
strip_15_test_no_labels  already exists
strip_16_test_no_labels  already exists
strip_17_test_no_labels  already exists
strip_18_test_no_labels  already exists
strip_19_test_no_labels  already exists
strip_20_test_no_labels  already exists
strip_21_test_no_labels  already exists
strip_22_test_no_labels  already exists
strip_23_test_no_labels  already exists


In [3]:
#Load test data
testDataSets = ['Empty']

for i in range(1, 24):
    if i == 1 or i == 22 or i == 23:
        print("DataSet " + str(i) + " | Skip zero dataset")
        testDataSets.append('Empty')
        continue
    df = npd.load_strip_pkl('%sstrip_%i_test_no_labels.pkl' % (npd.output_dir_test, i))

    df = npd.split_data(df, True)

    # fill nan
    columns = ["ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "r"] # "strip_id", "timestamp", 
    newColumns = range(1,16)
    for c in columns:
        cols = list(map(lambda x: c+str(x), newColumns))
        for c2 in cols:
            df[c2].fillna(df[c2].mean() if not math.isnan(df[c2].mean()) else 0, inplace=True)

    X_test = df

    sc = StandardScaler()
    sc.fit(X_test)
    X_test_transformed = sc.transform(X_test)
    testDataSets.append(X_test_transformed)

DataSet 1 | Skip zero dataset
DataSet 22 | Skip zero dataset
DataSet 23 | Skip zero dataset


In [4]:
#Load classification models
testClassificationModels = ['Empty']
for i in range(1, 24):
    if i == 1 or i == 22 or i == 23:
            print("CModel " + str(i) + " | Skip zero model")
            testClassificationModels.append('Empty')
            continue
    model = npd.readClassificationModel(i)
    testClassificationModels.append(model)

CModel 1 | Skip zero model
CModel 22 | Skip zero model
CModel 23 | Skip zero model


In [5]:
#Load regression models
testRegressionModels = ['Empty']
for i in range(1, 24):
    if i == 1 or i == 22 or i == 23:
        print("RModel " + str(i) + " | Skip zero model")
        testRegressionModels.append('Empty')
        continue
    model = npd.readRegressionModel(i)
    testRegressionModels.append(model)   

RModel 1 | Skip zero model
RModel 22 | Skip zero model
RModel 23 | Skip zero model


In [6]:
def predict_strip(): 
    frames = len(testDataSets[1])
    for frame in range(0, frames):
        framePrediction = np.zeros(23, dtype=np.int32)
        for strip in range(1, 24):
            if strip == 1 or strip == 22 or strip == 23:
                framePrediction[strip - 1] = 0
                continue
            #Get model for target strip
            targetModel = testClassificationModels[strip-1]

In [7]:
##################
#Iterate over all frames and create prediction
positionPrediction = np.zeros((int(3412),2), dtype=np.float64)
lastPositiveStrip = None
predictedCaseOne = 0
predictedCaseTwo = 0
predictedCaseThree = 0
for frame in range(0, 3412):

    #Prediction is stored as an array
    #each entry represents the prediction of one strip
    framePrediction = np.zeros(23, dtype=np.int32)

    #Iterate over all strips
    for strip in range(1, 24):
        if strip == 1 or strip == 22 or strip == 23:
            framePrediction[strip - 1] = 0
            continue
        #Get model for target strip
        targetModel = testClassificationModels[strip]

        #Get frame for target strip
        targetFrameData = testDataSets[strip][frame]

        #Create prediction for frame
        prediction = targetModel.predict([targetFrameData])
        framePrediction[strip - 1] = prediction[0]

    print("Predicted for frame {:04d}".format(frame) + " \t" + str(framePrediction), end = '', flush=True)

    #Check which prediction case we have
    #Case 1 : Only one positive prediction
    #Case 2 : No positive prediction
    #Case 3 : Multiple positive predictions

    positiveCount = np.count_nonzero(framePrediction == 1)
    if positiveCount == 1:
        #Case 1
        #Predict position with regressor on only one strip
        print(" case 1", end = '', flush=True)
        targetStripIndices = np.where(framePrediction == 1)

        #+1 because array index 0 is strip 1
        targetStripIndex = (int(targetStripIndices[0][0])) + 1

        #Check if position is realistic
        if lastPositiveStrip is not None:
            diff = abs(lastPositiveStrip - targetStripIndex)
            if diff >= 3: 
                #Position is unrealistic
                positionPrediction[frame][0] = positionPrediction[frame - 1][0]
                positionPrediction[frame][1] = positionPrediction[frame - 1][1]
                print(" predicted " + str(positionPrediction[frame]))
                lastPositiveStrip = None
                predictedCaseOne = predictedCaseOne + 1
                continue

        targetModel = testRegressionModels[targetStripIndex]
        targetFrameData = testDataSets[targetStripIndex][frame]
        prediction = targetModel.predict([targetFrameData])

        #Store prediction
        positionPrediction[frame][0] = prediction[0][0]
        positionPrediction[frame][1] = prediction[0][1]

        print(" predicted " + str(positionPrediction[frame]))

        lastPositiveStrip = targetStripIndex

        predictedCaseOne = predictedCaseOne + 1

    elif positiveCount == 0:
        #Case 2
        print(" case 2", end = '', flush=True)
        #TODO just use prev prediction

        #Predict next frame and calculate mean

        #Write zero and calculate mean later
        #positionPrediction[frame][0] = positionPrediction[frame - 1][0]
        #positionPrediction[frame][1] = positionPrediction[frame - 1][1]

        positionPrediction[frame][0] = 0
        positionPrediction[frame][1] = 0

        print(" predicted " + str(positionPrediction[frame]))

        lastPositiveStrip = None

        predictedCaseTwo = predictedCaseTwo + 1
    elif positiveCount > 1:
        #Case 3
        print(" case 3", end = '', flush=True)

        targetStripIndices = np.where(framePrediction == 1)

        #If the predictions are far apart, we only use the prediction that is closer to prev prediction
        #Sort desceding
        targetStripIndices[0][::-1].sort()
        diff = targetStripIndices[0][0] + targetStripIndices[0][0]
        for arrayIndex in range(len(targetStripIndices[0])):
            #print("index " + str(targetStripIndices[0][arrayIndex]))
            diff = diff - targetStripIndices[0][arrayIndex]

        diff = abs(diff)

        print(" diff " + str(diff), end = '', flush=True)

        #Predict for all positive results and calculate mean
        if diff <= 2:
            
            finalPrediction = np.zeros((2), dtype=np.float64)
            for arrayIndex in range(len(targetStripIndices[0])):
                targetStripIndex = (int(targetStripIndices[0][arrayIndex])) + 1

                targetModel = testRegressionModels[targetStripIndex]
                targetFrameData = testDataSets[targetStripIndex][frame]
                prediction = targetModel.predict([targetFrameData])

                #Store prediction
                finalPrediction[0] = finalPrediction[0] + prediction[0][0]
                finalPrediction[1] = finalPrediction[1] + prediction[0][1]

            #Calulate mean value
            finalPrediction[0] = finalPrediction[0] / positiveCount
            finalPrediction[1] = finalPrediction[1] / positiveCount

            positionPrediction[frame][0] = finalPrediction[0]
            positionPrediction[frame][1] = finalPrediction[1]

        #Predictions are too far away, so we use the pre prediction
        else:
            #TODO caluclate closest to prev
            positionPrediction[frame][0] = positionPrediction[frame - 1][0]
            positionPrediction[frame][1] = positionPrediction[frame - 1][1]

        print(" predicted " + str(positionPrediction[frame]))

        lastPositiveStrip = None

        predictedCaseThree = predictedCaseThree + 1

#Run over all predictions and fill zero values with mean between previous and next frame
#TODO handle if multiple successively frames are zero (interpolate)
for frame in range(len(positionPrediction)):
    if positionPrediction[frame][0] == 0 and positionPrediction[frame][1] == 0 and frame != 0 and frame != 3412:

        #If next frame is also zero, we just copy the prev frame
        if positionPrediction[frame + 1][0] == 0 and positionPrediction[frame + 1][1] == 0:
            positionPrediction[frame][0] = positionPrediction[frame - 1][0]
            positionPrediction[frame][1] = positionPrediction[frame - 1][1]
        else:
            positionPrediction[frame][0] = (positionPrediction[frame - 1][0] + positionPrediction[frame + 1][0]) / 2
            positionPrediction[frame][1] = (positionPrediction[frame - 1][1] + positionPrediction[frame + 1][1]) / 2

        print("Filled frame {:04d}".format(frame) + " \t with " + str(positionPrediction[frame]))


print("Prediction result")
print("case 1 " + str(predictedCaseOne))
print("case 2 " + str(predictedCaseTwo))
print("case 3 " + str(predictedCaseThree))

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1169 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0] case 1 predicted [8.83327253 4.77961035]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1170 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0] case 1 predicted [8.83402484 4.56309613]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1171 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0] case 1 predicted [8.84948331 4.52597553]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1172 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0] case 3 diff 1 predicted [8.38857705 3.36542187]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1173 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0] case 1 predicted [8.83351535 4.57859624]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Predicted for frame 1174 	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0] case 1 predicted [8.83915058 4.5302542

In [11]:
len(testRegressionModels)

21